In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_excel("sample tweets.xlsx")

In [3]:
data=data[data['Sentiment'] != 'Neutral'].reset_index(drop=True)

In [4]:
X=data['SentimentText']
Y=data['Sentiment']

In [5]:
with open('../glove.txt',encoding='mbcs') as f:
    doc=f.readlines()
    
glove_emb={
          }
for i in range(len(doc)):
    word   = doc[i].split(" ")[0]
    vector = np.array(doc[i].split(" ")[1:],dtype='float32')
    
    glove_emb[word]=vector

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
rg=RegexpTokenizer("[a-zA-Z]+")
sw=stopwords.words("english")

In [7]:
unique={
    
}
weight=[]
k=1;
X_train=np.zeros((X.shape[0],45))
X_train=X_train.astype('int32')
for i in range(X.shape[0]):
    words=rg.tokenize(X[i])
    words=[w for w in words if w not in sw and w in glove_emb and len(w)>2]
    
    for j in range(min(len(words),45)):
        if words[j].lower() in unique:
            X_train[i][j]=unique[words[j].lower()]
            
        else:
            weight.append(glove_emb[words[j]])
            X_train[i][j]=k
            unique[words[j].lower()]=k
            k+=1

In [8]:
weight_matrix=[np.zeros((50,))]+weight
weight_matrix=np.array(weight_matrix)

In [9]:
weight_matrix.shape

(3679, 50)

In [10]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
Y_train=le.fit_transform(Y)

In [11]:
from keras.models import Sequential
from keras.layers import *
from keras.models import Model

Using TensorFlow backend.


In [12]:
import keras.backend as K


In [13]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [14]:
inputs=Input((45,))
x=Embedding(3679,50, weights=[weight_matrix], input_length=45, trainable=False)(inputs)
att_in=LSTM(32,return_sequences=True,dropout=0.3,recurrent_dropout=0.2)(x)
att_out=attention()(att_in)
outputs=Dense(1,activation='sigmoid',trainable=True)(att_out)
model=Model(inputs,outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 45)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 45, 50)            183950    
_________________________________________________________________
lstm_1 (LSTM)                (None, 45, 32)            10624     
_________________________________________________________________
attention_1 (attention)      (None, 32)                77        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 194,684
Trainable params: 10,734
Non-trainable params: 183,950
_________________________________________________________________


In [15]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['acc'])

In [16]:
hist=model.fit(X_train,Y_train,validation_split=.2,epochs=15,batch_size=32)

Train on 5686 samples, validate on 1422 samples
Epoch 1/15
5686/5686 [==============================] - 5s 802us/step - loss: 0.2890 - acc: 0.9161 - val_loss: 0.2876 - val_acc: 0.9114
Epoch 2/15
5686/5686 [==============================] - 4s 645us/step - loss: 0.1917 - acc: 0.9370 - val_loss: 0.2772 - val_acc: 0.9135
Epoch 3/15
5686/5686 [==============================] - 3s 611us/step - loss: 0.1796 - acc: 0.9425 - val_loss: 0.2789 - val_acc: 0.9142
Epoch 4/15
5686/5686 [==============================] - 4s 620us/step - loss: 0.1662 - acc: 0.9483 - val_loss: 0.2834 - val_acc: 0.9128
Epoch 5/15
5686/5686 [==============================] - 3s 614us/step - loss: 0.1635 - acc: 0.9464 - val_loss: 0.2724 - val_acc: 0.9135
Epoch 6/15
5686/5686 [==============================] - 4s 633us/step - loss: 0.1573 - acc: 0.9513 - val_loss: 0.2654 - val_acc: 0.9142
Epoch 7/15
5686/5686 [==============================] - 4s 625us/step - loss: 0.1573 - acc: 0.9539 - val_loss: 0.2546 - val_acc: 0.9184


In [17]:
Y_pred=model.predict(X_train)

In [18]:
Y_pr=[]
for i in Y_pred:
    if i>0.5:
        Y_pr.append(1)
    else :
        Y_pr.append(0)

In [19]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train,Y_pr)

array([[6594,    5],
       [ 325,  184]], dtype=int64)

# with equal Positive and Negative Examples

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_excel("sample tweets.xlsx")

In [3]:
dataN=data[data['Sentiment'] == 'Negative']
dataP=data[data['Sentiment'] == 'Positive']
dataN=dataN.sample(509)
data=pd.concat([dataN,dataP])
data=data.reset_index(drop=True)

In [4]:
X=data['SentimentText']
Y=data['Sentiment']

In [5]:
with open('../glove.txt',encoding='mbcs') as f:
    doc=f.readlines()
    
glove_emb={
          }
for i in range(len(doc)):
    word   = doc[i].split(" ")[0]
    vector = np.array(doc[i].split(" ")[1:],dtype='float32')
    
    glove_emb[word]=vector

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
rg=RegexpTokenizer("[a-zA-Z]+")
sw=stopwords.words("english")

In [7]:
unique={
    
}
weight=[]
k=1;
X_train=np.zeros((X.shape[0],45))
X_train=X_train.astype('int32')
for i in range(X.shape[0]):
    words=rg.tokenize(X[i])
    words=[w for w in words if w not in sw and w in glove_emb and len(w)>2]
    
    for j in range(min(len(words),45)):
        if words[j].lower() in unique:
            X_train[i][j]=unique[words[j].lower()]
            
        else:
            weight.append(glove_emb[words[j]])
            X_train[i][j]=k
            unique[words[j].lower()]=k
            k+=1

In [8]:
weight_matrix=[np.zeros((50,))]+weight
weight_matrix=np.array(weight_matrix)

In [9]:
weight_matrix.shape

(1641, 50)

In [10]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
Y_train=le.fit_transform(Y)

In [11]:
from keras.models import Sequential
from keras.layers import *
from keras.models import Model,load_model
from keras.callbacks import ModelCheckpoint,EarlyStopping

Using TensorFlow backend.


In [12]:
import keras.backend as K

In [13]:
modelcheckpoint=ModelCheckpoint("./best_model.h5",monitor="val_acc",verbose=1,save_best_only=True)
# early_stop=EarlyStopping(monitor='val_acc',min_delta=0.001,patience=10,verbose=1,restore_best_weights=True)
call_back=[modelcheckpoint]

In [14]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [15]:
inputs=Input((45,))
x=Embedding(weight_matrix.shape[0],50, weights=[weight_matrix], input_length=45, trainable=False)(inputs)
att_in=LSTM(32,return_sequences=True,dropout=0.3,recurrent_dropout=0.2)(x)
att_out=attention()(att_in)
outputs=Dense(1,activation='sigmoid',trainable=True)(att_out)
model=Model(inputs,outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 45)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 45, 50)            82050     
_________________________________________________________________
lstm_1 (LSTM)                (None, 45, 32)            10624     
_________________________________________________________________
attention_1 (attention)      (None, 32)                77        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 92,784
Trainable params: 10,734
Non-trainable params: 82,050
_________________________________________________________________


In [16]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['acc'])

In [17]:
hist=model.fit(X_train,Y_train,validation_split=.2,epochs=150,batch_size=32,callbacks=call_back)

Train on 814 samples, validate on 204 samples
Epoch 1/150
814/814 [==============================] - 1s 1ms/step - loss: 0.6601 - acc: 0.6081 - val_loss: 0.9316 - val_acc: 0.0000e+00

Epoch 00001: val_acc improved from -inf to 0.00000, saving model to ./best_model.h5
Epoch 2/150
814/814 [==============================] - 1s 650us/step - loss: 0.6058 - acc: 0.6253 - val_loss: 1.0153 - val_acc: 0.0000e+00

Epoch 00002: val_acc did not improve from 0.00000
Epoch 3/150
814/814 [==============================] - 1s 658us/step - loss: 0.5465 - acc: 0.7310 - val_loss: 1.1168 - val_acc: 0.5637

Epoch 00003: val_acc improved from 0.00000 to 0.56373, saving model to ./best_model.h5
Epoch 4/150
814/814 [==============================] - 1s 686us/step - loss: 0.4957 - acc: 0.7813 - val_loss: 1.0054 - val_acc: 0.6275

Epoch 00004: val_acc improved from 0.56373 to 0.62745, saving model to ./best_model.h5
Epoch 5/150
814/814 [==============================] - 1s 722us/step - loss: 0.4865 - acc: 0.788

814/814 [==============================] - 1s 679us/step - loss: 0.2947 - acc: 0.8870 - val_loss: 0.8783 - val_acc: 0.6667

Epoch 00044: val_acc did not improve from 0.67157
Epoch 45/150
814/814 [==============================] - 1s 774us/step - loss: 0.2940 - acc: 0.8845 - val_loss: 1.0081 - val_acc: 0.5049

Epoch 00045: val_acc did not improve from 0.67157
Epoch 46/150
814/814 [==============================] - 1s 741us/step - loss: 0.3016 - acc: 0.8686 - val_loss: 0.9950 - val_acc: 0.6814

Epoch 00046: val_acc improved from 0.67157 to 0.68137, saving model to ./best_model.h5
Epoch 47/150
814/814 [==============================] - 1s 684us/step - loss: 0.3006 - acc: 0.8784 - val_loss: 1.0225 - val_acc: 0.5147

Epoch 00047: val_acc did not improve from 0.68137
Epoch 48/150
814/814 [==============================] - 1s 709us/step - loss: 0.2922 - acc: 0.8833 - val_loss: 0.8561 - val_acc: 0.6912

Epoch 00048: val_acc improved from 0.68137 to 0.69118, saving model to ./best_model.h5
Epoc


Epoch 00086: val_acc did not improve from 0.72549
Epoch 87/150
814/814 [==============================] - 1s 718us/step - loss: 0.2346 - acc: 0.8980 - val_loss: 1.1924 - val_acc: 0.6618

Epoch 00087: val_acc did not improve from 0.72549
Epoch 88/150
814/814 [==============================] - 1s 715us/step - loss: 0.2207 - acc: 0.9054 - val_loss: 1.2801 - val_acc: 0.5196

Epoch 00088: val_acc did not improve from 0.72549
Epoch 89/150
814/814 [==============================] - 1s 758us/step - loss: 0.2150 - acc: 0.9103 - val_loss: 1.0629 - val_acc: 0.6912

Epoch 00089: val_acc did not improve from 0.72549
Epoch 90/150
814/814 [==============================] - 1s 824us/step - loss: 0.2100 - acc: 0.9079 - val_loss: 1.1866 - val_acc: 0.6765

Epoch 00090: val_acc did not improve from 0.72549
Epoch 91/150
814/814 [==============================] - 1s 663us/step - loss: 0.2378 - acc: 0.8907 - val_loss: 0.9351 - val_acc: 0.7010

Epoch 00091: val_acc did not improve from 0.72549
Epoch 92/150
8

814/814 [==============================] - 1s 676us/step - loss: 0.1530 - acc: 0.9361 - val_loss: 1.3348 - val_acc: 0.6961

Epoch 00130: val_acc did not improve from 0.75490
Epoch 131/150
814/814 [==============================] - 1s 756us/step - loss: 0.1901 - acc: 0.9201 - val_loss: 1.2227 - val_acc: 0.6912

Epoch 00131: val_acc did not improve from 0.75490
Epoch 132/150
814/814 [==============================] - 1s 700us/step - loss: 0.1602 - acc: 0.9386 - val_loss: 1.3515 - val_acc: 0.6716

Epoch 00132: val_acc did not improve from 0.75490
Epoch 133/150
814/814 [==============================] - 1s 727us/step - loss: 0.1830 - acc: 0.9275 - val_loss: 1.3490 - val_acc: 0.6716

Epoch 00133: val_acc did not improve from 0.75490
Epoch 134/150
814/814 [==============================] - 1s 727us/step - loss: 0.1693 - acc: 0.9312 - val_loss: 1.1570 - val_acc: 0.7157

Epoch 00134: val_acc did not improve from 0.75490
Epoch 135/150
814/814 [==============================] - 1s 666us/step - l

In [18]:
new_model=load_model("best_model.h5", custom_objects={'attention': attention})

In [19]:
Y_pred=new_model.predict(X_train)

In [20]:
Y_pr=[]
for i in Y_pred:
    if i>=0.5:
        Y_pr.append(1)
    else :
        Y_pr.append(0)

In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train,Y_pr)

array([[464,  45],
       [ 56, 453]], dtype=int64)